# Transfer Learning

In this notebook, you'll learn how to use pre-trained networks to solved challenging problems in computer vision. Specifically, you'll use networks trained on [ImageNet](http://www.image-net.org/) [available from torchvision](http://pytorch.org/docs/0.3.0/torchvision/models.html). 

ImageNet is a massive dataset with over 1 million labeled images in 1000 categories. It's used to train deep neural networks using an architecture called convolutional layers. I'm not going to get into the details of convolutional networks here, but if you want to learn more about them, please [watch this](https://www.youtube.com/watch?v=2-Ol7ZB0MmU).

Once trained, these models work astonishingly well as feature detectors for images they weren't trained on. Using a pre-trained network on images not in the training set is called transfer learning. Here we'll use transfer learning to train a network that can classify our cat and dog photos with near perfect accuracy.

With `torchvision.models` you can download these pre-trained networks and use them in your applications. We'll include `models` in our imports now.

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

Most of the pretrained models require the input to be 224x224 images. Also, we'll need to match the normalization used when the models were trained. Each color channel was normalized separately, the means are `[0.485, 0.456, 0.406]` and the standard deviations are `[0.229, 0.224, 0.225]`.

In [2]:
data_dir = 'Cat_Dog_data/Cat_Dog_data'

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                      transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                               transforms.CenterCrop(224),
                               transforms.ToTensor(),
                               transforms.Normalize([0.485, 0.456, 0.406],
                                                    [.229, 0.224, 0.225])]) 

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

We can load in a model such as [DenseNet](http://pytorch.org/docs/0.3.0/torchvision/models.html#id5). Let's print out the model architecture so we can see what's going on.

In [3]:
model = models.densenet121(pretrained=True)
model

/home/user/miniconda/envs/py36/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

This model is built out of two main parts, the features and the classifier. The features part is a stack of convolutional layers and overall works as a feature detector that can be fed into a classifier. The classifier part is a single fully-connected layer `(classifier): Linear(in_features=1024, out_features=1000)`. This layer was trained on the ImageNet dataset, so it won't work for our specific problem. That means we need to replace the classifier, but the features will work perfectly on their own. In general, I think about pre-trained networks as amazingly good feature detectors that can be used as the input for simple feed-forward classifiers.

In [4]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

With our model built, we need to train the classifier. However, now we're using a **really deep** neural network. If you try to train this on a CPU like normal, it will take a long, long time. Instead, we're going to use the GPU to do the calculations. The linear algebra computations are done in parallel on the GPU leading to 100x increased training speeds. It's also possible to train on multiple GPUs, further decreasing training time.

PyTorch, along with pretty much every other deep learning framework, uses [CUDA](https://developer.nvidia.com/cuda-zone) to efficiently compute the forward and backwards passes on the GPU. In PyTorch, you move your model parameters and other tensors to the GPU memory using `model.to('cuda')`. You can move them back from the GPU with `model.to('cpu')` which you'll commonly do when you need to operate on the network output outside of PyTorch. As a demonstration of the increased speed, I'll compare how long it takes to perform a forward and backward pass with and without a GPU.

In [5]:
import time

In [7]:
for device in ['cpu', 'cuda']:

    criterion = nn.NLLLoss()
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    model.to(device)

    for ii, (inputs, labels) in enumerate(trainloader):

        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        start = time.time()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if ii==3:
            break
        
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

Device = cpu; Time per batch: 11.147 seconds
Device = cuda; Time per batch: 0.014 seconds


You can write device agnostic code which will automatically use CUDA if it's enabled like so:
```python
# at beginning of the script
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

...

# then whenever you get a new Tensor or Module
# this won't copy if they are already on the desired device
input = data.to(device)
model = MyModule(...).to(device)
```

From here, I'll let you finish training the model. The process is the same as before except now your model is much more powerful. You should get better than 95% accuracy easily.

>**Exercise:** Train a pretrained models to classify the cat and dog images. Continue with the DenseNet model, or try ResNet, it's also a good model to try out first. Make sure you are only training the classifier and the parameters for the features part are frozen.

#### Resnet101

Load the ResNet101 model.

In [24]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device {device}")

model = models.resnet101(pretrained=True)
model

Device cuda:0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

Freeze parameters.

In [25]:
for param in model.parameters():
    param.requires_grad = False

Replace model head.

In [26]:
model.fc = nn.Sequential(nn.Linear(2048, 256),
                        nn.ReLU(),
                        nn.Dropout(0.2),
                        nn.Linear(256, 2),
                        nn.LogSoftmax(dim=1))

In [27]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

Define the loss function and the optimizer.

In [28]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters())

Training loop.

In [29]:
n_epochs = 1
print_every = 5
step, train_loss, train_acc, test_loss, test_acc = 0, 0, 0, 0, 0

def get_batch_acc(preds, labels):
    _, pred_class = preds.topk(1, dim=1)
    correct_class = (pred_class == labels.view(*pred_class.shape))
    batch_acc = torch.mean(correct_class.type(torch.FloatTensor))
    return batch_acc

for epoch in range(n_epochs):
    for images, labels in trainloader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        # forward pass
        preds = model(images)
        # compute loss & accuracy
        batch_loss = criterion(preds, labels)
        batch_acc = get_batch_acc(preds, labels)
        # backward pass
        batch_loss.backward()
        # gradient descent step
        optimizer.step()
        # update running loss and accuracy
        train_loss += batch_loss.item() / print_every
        train_acc += batch_acc.item() / print_every
        
        if step % print_every == 0:
            # Evaluate on test data
            with torch.no_grad():
                model.eval()
                for test_images, test_labels in testloader:
                    test_images, test_labels = test_images.to(device), test_labels.to(device)
                    test_preds = model(test_images)
                    batch_loss = criterion(test_preds, test_labels)
                    batch_acc = get_batch_acc(test_preds, test_labels)
                    test_loss += batch_loss.item() / len(testloader)
                    test_acc += batch_acc.item() / len(testloader)
            print(f"Epoch: {epoch} | Step: {step}/{int(len(trainloader) / print_every)} | Train loss: {train_loss} | Train acc: {train_acc} | Test loss: {test_loss} | Test acc: {test_acc}")
            model.train()
            train_loss, train_acc, test_loss, test_acc = 0, 0, 0, 0
            
        step += 1

Epoch: 0 | Step: 0/70 | Train loss: 0.1420997977256775 | Train acc: 0.09375 | Test loss: 0.5888391725718976 | Test acc: 0.5136718750000002
Epoch: 0 | Step: 5/70 | Train loss: 0.6307643175125123 | Train acc: 0.628125 | Test loss: 0.23891626708209512 | Test acc: 0.9695312500000004
Epoch: 0 | Step: 10/70 | Train loss: 0.28028063476085663 | Train acc: 0.9125 | Test loss: 0.12141264081001284 | Test acc: 0.9765625000000002
Epoch: 0 | Step: 15/70 | Train loss: 0.24202772676944734 | Train acc: 0.909375 | Test loss: 0.17700428750831634 | Test acc: 0.9289062500000004
Epoch: 0 | Step: 20/70 | Train loss: 0.24063588082790371 | Train acc: 0.8937499999999999 | Test loss: 0.12248691681306804 | Test acc: 0.9554687500000005
Epoch: 0 | Step: 25/70 | Train loss: 0.17640648782253265 | Train acc: 0.934375 | Test loss: 0.060022399388253704 | Test acc: 0.9773437500000003
Epoch: 0 | Step: 30/70 | Train loss: 0.18905198574066162 | Train acc: 0.9124999999999999 | Test loss: 0.05537798898294568 | Test acc: 0.978

Epoch: 0 | Step: 280/70 | Train loss: 0.10014800429344178 | Train acc: 0.9562499999999999 | Test loss: 0.05499271389562638 | Test acc: 0.9773437500000002
Epoch: 0 | Step: 285/70 | Train loss: 0.18766720890998842 | Train acc: 0.91875 | Test loss: 0.08434993200935428 | Test acc: 0.9671875000000005
Epoch: 0 | Step: 290/70 | Train loss: 0.13800379261374474 | Train acc: 0.940625 | Test loss: 0.03574889963492751 | Test acc: 0.9867187500000005
Epoch: 0 | Step: 295/70 | Train loss: 0.15387942492961884 | Train acc: 0.9187500000000002 | Test loss: 0.03619727061595768 | Test acc: 0.9863281250000006
Epoch: 0 | Step: 300/70 | Train loss: 0.1250557154417038 | Train acc: 0.9625 | Test loss: 0.038927224534563716 | Test acc: 0.9863281250000003
Epoch: 0 | Step: 305/70 | Train loss: 0.16261594146490096 | Train acc: 0.9375 | Test loss: 0.03769625669810921 | Test acc: 0.9871093750000005
Epoch: 0 | Step: 310/70 | Train loss: 0.09066415578126907 | Train acc: 0.965625 | Test loss: 0.03774670073762536 | Test a

#### Densenet121

In [8]:
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.densenet121(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False
    
model.classifier = nn.Sequential(nn.Linear(1024, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(256, 2),
                                 nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

model.to(device);

In [9]:
device

device(type='cuda')

In [10]:
## TODO: Use a pretrained model to classify the cat and dog images

epochs = 1
steps = 0
running_loss = 0
print_every = 5
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()

Epoch 1/1.. Train loss: 1.046.. Test loss: 0.468.. Test accuracy: 0.691
Epoch 1/1.. Train loss: 0.476.. Test loss: 0.286.. Test accuracy: 0.880
Epoch 1/1.. Train loss: 0.347.. Test loss: 0.218.. Test accuracy: 0.920
Epoch 1/1.. Train loss: 0.316.. Test loss: 0.100.. Test accuracy: 0.976
Epoch 1/1.. Train loss: 0.262.. Test loss: 0.088.. Test accuracy: 0.977
Epoch 1/1.. Train loss: 0.287.. Test loss: 0.144.. Test accuracy: 0.942
Epoch 1/1.. Train loss: 0.187.. Test loss: 0.094.. Test accuracy: 0.970
Epoch 1/1.. Train loss: 0.263.. Test loss: 0.063.. Test accuracy: 0.980
Epoch 1/1.. Train loss: 0.189.. Test loss: 0.100.. Test accuracy: 0.963
Epoch 1/1.. Train loss: 0.176.. Test loss: 0.061.. Test accuracy: 0.979
Epoch 1/1.. Train loss: 0.186.. Test loss: 0.056.. Test accuracy: 0.980
Epoch 1/1.. Train loss: 0.142.. Test loss: 0.061.. Test accuracy: 0.978
Epoch 1/1.. Train loss: 0.131.. Test loss: 0.050.. Test accuracy: 0.982
Epoch 1/1.. Train loss: 0.178.. Test loss: 0.049.. Test accuracy